## Votação Seção


```mysql
create table votacao_secao_2016(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    CODIGO_MUNICIPIO STRING,
    NOME_MUNICIPIO STRING,
    NUM_ZONA STRING,
    NUM_SECAO STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NUM_VOTAVEL STRING,
    QTDE_VOTOS STRING) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\,',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table votacao_secao_2016 set location 's3://datascience2016/data/gzip/votacao_secao/2016/';
```



## Candidato

```mysql
create table candidato_2016 
    (DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    DESCRICAO_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NOME_CANDIDATO STRING,
    SEQUENCIAL_CANDIDATO STRING,
    NUMERO_CANDIDATO STRING,
    CPF_CANDIDATO STRING,
    NOME_URNA_CANDIDATO STRING,
    COD_SITUACAO_CANDIDATURA STRING,
    DES_SITUACAO_CANDIDATURA STRING,
    NUMERO_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    CODIGO_LEGENDA STRING,
    SIGLA_LEGENDA STRING,
    COMPOSICAO_LEGENDA STRING,
    NOME_LEGENDA STRING,
    CODIGO_OCUPACAO STRING,
    DESCRICAO_OCUPACAO STRING,
    DATA_NASCIMENTO STRING,
    NUM_TITULO_ELEITORAL_CANDIDATO STRING,
    IDADE_DATA_ELEICAO STRING,
    CODIGO_SEXO STRING,
    DESCRICAO_SEXO STRING,
    COD_GRAU_INSTRUCAO STRING,
    DESCRICAO_GRAU_INSTRUCAO STRING,
    CODIGO_ESTADO_CIVIL STRING,
    DESCRICAO_ESTADO_CIVIL STRING,
    CODIGO_COR_RACA STRING,
    DESCRICAO_COR_RACA STRING,
    CODIGO_NACIONALIDADE STRING,
    DESCRICAO_NACIONALIDADE STRING,
    SIGLA_UF_NASCIMENTO STRING,
    CODIGO_MUNICIPIO_NASCIMENTO STRING,
    NOME_MUNICIPIO_NASCIMENTO STRING,
    DESPESA_MAX_CAMPANHA STRING,
    COD_SIT_TOT_TURNO STRING,
    DESC_SIT_TOT_TURNO STRING,
    NM_EMAIL STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table candidato set location 's3://datascience2016/data/gzip/candidatos/candidato_2016.csv.gz';
```


### Legenda


```mysql
create table legenda_2016(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    NOME_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    TIPO_LEGENDA STRING,
    NUM_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    SIGLA_COLIGACAO STRING,
    NOME_COLIGACAO STRING,
    COMPOSICAO_COLIGACAO STRING,
    SEQUENCIA_COLIGACAO STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"'
)
STORED AS TEXTFILE;
Alter table legenda_2016 set location 's3://datascience2016/data/gzip/legendas/legendas_2016.csv.gz';
```





## Candidato - legenda
```mysql
    select 
    * 
    from candidato_2016 as c 
    join legenda_2016 as l 
    on c.SIGLA_UE=l.SIGLA_UE 
    AND c.NUMERO_PARTIDO=l.NUM_PARTIDO 
    AND c.CODIGO_CARGO=l.CODIGO_CARGO
```

# Agregações Regionais


### macro
```mysql
create view votacao_secao_2016_macro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
sum(cast(QTDE_VOTOS AS INT)) as total_de_votos
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO
from votacao_secao_2016 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO;

```
### meso

```mysql
create view votacao_secao_2016_meso as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
sum(cast(QTDE_VOTOS AS INT)) 
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.CODIGO_MESO,m.NOME_MESO 
from votacao_secao_2016 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO;
```
### micro

```mysql
create view votacao_secao_2016_micro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
sum(cast(QTDE_VOTOS AS INT)) 
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO 
from votacao_secao_2016 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO;

```

### UF
```mysql
create view votacao_secao_2016_uf as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
sum(cast(QTDE_VOTOS AS INT)) 
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF 
from votacao_secao_2016 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF;

```


## join com os candidatos

```mysql

select * from votacao_secao_2016_macro as v
join candidato_2016
create view votacao_secao_2016_macro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
sum(cast(QTDE_VOTOS AS INT)) 
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO
from votacao_secao_2016 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO;
```



select m.*,v.total_de_votos
from votacao_secao_2016_macro as v 
join candidato_2016 as m on 
v.ANO_ELEICAO=m.ANO_ELEICAO 
AND v.NUM_TURNO=m.NUM_TURNO
AND v.CODIGO_CARGO=m.CODIGO_CARGO
AND v.NUM_VOTAVEL=m.NUMERO_CANDIDATO;